In [1]:
import pandas as pd

### Load data

In [2]:
memcache = pd.read_csv('memcache.csv', index_col=0)
memcache.head()

,rdma,tcp,nop
0,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
1,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
2,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
3,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
4,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...


### parser code

In [38]:
entry = memcache['tcp'][0]

def init():
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    cats_index = {'Sets':9, 'Gets':10, 'Totals':12}
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[1])
        res[cat]['Avg. Latency'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[4])
        res[cat]['p50 Latency'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[5])
        res[cat]['p90 Latency'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[6])
        res[cat]['p99 Latency'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[7])
        res[cat]['KB/sec'] = 0 #float(entry.split('\n')[cats_index[cat]].split()[8])
    return res 
def extract(entry):
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    cats_index = {'Sets':9, 'Gets':10, 'Totals':12}
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = float(entry.split('\n')[cats_index[cat]].split()[1])
        res[cat]['Avg. Latency'] = float(entry.split('\n')[cats_index[cat]].split()[4])
        res[cat]['p50 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[5])
        res[cat]['p90 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[6])
        res[cat]['p99 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[7])
        res[cat]['KB/sec'] = float(entry.split('\n')[cats_index[cat]].split()[8])
    return res 

def add(res1, res2):
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = res1[cat]['Ops/sec'] + res2[cat]['Ops/sec']
        res[cat]['Avg. Latency'] = res1[cat]['Avg. Latency'] + res2[cat]['Avg. Latency']
        res[cat]['p50 Latency'] = res1[cat]['p50 Latency'] + res2[cat]['p50 Latency']
        res[cat]['p90 Latency'] = res1[cat]['p90 Latency'] + res2[cat]['p90 Latency']
        res[cat]['p99 Latency'] = res1[cat]['p99 Latency'] + res2[cat]['p99 Latency']
        res[cat]['KB/sec'] = res1[cat]['KB/sec'] + res2[cat]['KB/sec']
    return res 
def scale(res, n):
    cats = ['Sets', 'Gets', 'Totals']
    for cat in cats:
        res[cat]['Ops/sec'] /= n
        res[cat]['Avg. Latency'] /= n
        res[cat]['p50 Latency'] /= n
        res[cat]['p90 Latency'] /= n 
        res[cat]['p99 Latency'] /= n 
        res[cat]['KB/sec'] /= n
    return res 
extract(entry)


{'Sets': {'Ops/sec': 138.19,
  'Avg. Latency': 0.66273,
  'p50 Latency': 0.671,
  'p90 Latency': 0.831,
  'p99 Latency': 0.871,
  'KB/sec': 9.43},
 'Gets': {'Ops/sec': 1380.36,
  'Avg. Latency': 0.65372,
  'p50 Latency': 0.655,
  'p90 Latency': 0.807,
  'p99 Latency': 0.831,
  'KB/sec': 34.9},
 'Totals': {'Ops/sec': 1518.55,
  'Avg. Latency': 0.65454,
  'p50 Latency': 0.655,
  'p90 Latency': 0.807,
  'p99 Latency': 0.847,
  'KB/sec': 44.33}}

### RDMA vs TCP vs NoProxy

In [40]:
cases = ['rdma','tcp', 'nop']
for case in cases:
    res = init()
    xps = memcache[case]
    for xp in xps:
        res_xp = extract(xp)
        res = add(res, res_xp)
    scale(res, len(xps))
    print(case,'\n',res,'\n')
        

rdma 
 {'Sets': {'Ops/sec': 353.8639999999999, 'Avg. Latency': 0.26571599999999995, 'p50 Latency': 0.2502, 'p90 Latency': 0.391, 'p99 Latency': 0.399, 'KB/sec': 24.142000000000003}, 'Gets': {'Ops/sec': 3534.7340000000004, 'Avg. Latency': 0.253994, 'p50 Latency': 0.24219999999999997, 'p90 Latency': 0.3798, 'p99 Latency': 0.39740000000000003, 'KB/sec': 89.36800000000001}, 'Totals': {'Ops/sec': 3888.596, 'Avg. Latency': 0.255058, 'p50 Latency': 0.24219999999999997, 'p90 Latency': 0.3894, 'p99 Latency': 0.399, 'KB/sec': 113.51000000000002}} 

tcp 
 {'Sets': {'Ops/sec': 125.56199999999998, 'Avg. Latency': 0.735676, 'p50 Latency': 0.743, 'p90 Latency': 0.9030000000000001, 'p99 Latency': 0.9398, 'KB/sec': 8.568}, 'Gets': {'Ops/sec': 1254.216, 'Avg. Latency': 0.7215680000000001, 'p50 Latency': 0.7285999999999999, 'p90 Latency': 0.8806, 'p99 Latency': 0.9141999999999999, 'KB/sec': 31.71}, 'Totals': {'Ops/sec': 1379.776, 'Avg. Latency': 0.7228519999999999, 'p50 Latency': 0.7318, 'p90 Latency': 0

In [44]:
print(memcache['rdma'][0])
print(memcache['rdma'][1])
print(memcache['nop'][0])
print(memcache['nop'][1])

1         Threads
1         Connections per thread
10000     Requests per client


ALL STATS
Type         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec 
----------------------------------------------------------------------------------------------------------------------------
Sets          347.37          ---          ---         0.27016         0.25500         0.39100         0.39900        23.70 
Gets         3469.84         0.00      3469.84         0.25858         0.24700         0.38300         0.39900        87.73 
Waits           0.00          ---          ---             ---             ---             ---             ---          --- 
Totals       3817.21         0.00      3469.84         0.25963         0.24700         0.39100         0.39900       111.43 


Request Latency Distribution
Type     <= msec         Percent
------------------------------------------------------------------------
SET       0.191     